<a href="https://colab.research.google.com/github/YujiSue/BioInfoTools/blob/master/%E3%82%B3%E3%83%9E%E3%83%B3%E3%83%89%E4%B8%80%E8%A6%A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NGS解析用コマンド生成ノート (For UNIX/Linux/MacOS)
* このノートは、次世代シーケンス (NGS) のデータ解析でよく使われるソフトのコマンドを簡単に生成するためのノートです。  
<font color="red">※解析を実行するノートではありません。Google Colab上でNGSデータ解析を実行したい方は[こちらのノート](https://colab.research.google.com/drive/1evNMlFYZITvpUBcFFENpz5mjnJKeK-kX)を参照してください。</font>
* 実際にシーケンスした結果ファイルやリファレンス、パラメータなどを適宜自分の環境に合わせて書き換えて、実行（▶）するとコマンドが出力されます。  
出力されたコマンドを解析用PCのターミナル画面にコピペして使用してください。  

* 「既存のパイプラインで出来ない解析をマニュアルでやりたいけど、このソフトどんなコマンドを打てばいいんだっけ？」と思ったときに活用してみてください。
基本的にバイオインフォ初心者や、まだ慣れていない学生などが対象です。

## IonTorrentSuite (ThermoFisherScientific) 関連

### Run TMAP

In [ ]:
#@title 実行するとコマンド生成

リファレンスファイルのパス = '~/Reference/reference.fasta' #@param {type:"string"}
シーケンスファイルのパス = '~/Test/seq.fq' #@param {type:"string"}
シーケンスファイルのタイプ = 'fastq' #@param ['fastq','fq','fasta','fa','sam','bam']
cmd = 'tmap mapall'+\
' -r '+リファレンスファイルのパス+\
' -i '+シーケンスファイルのタイプ+\
' -f '+シーケンスファイルのパス+\
' -v -Y -u --prefix-exclude 5 -o 2  stage1 map4 2'
print(cmd)

tmap mapall -r ~/Reference/reference.fasta -i fastq -f ~/Test/seq.fq -v -Y -u --prefix-exclude 5 -o 2  stage1 map4 2


### Run VariantCall

In [3]:
#@title 実行するとコマンド生成
スレッド数 = 8 #@param {type:"number"}
リファレンスファイル = '~/Reference/reference.fasta' #@param {type:"string"}
パラメータ類のある場所 = '~/Preferences/TSVC' #@param {type:"string"}
パラメータファイル名 = 'parameters.json' #@param {type:"string"}
エラーモチーフのある場所 = '~/Preferences/TSVC' #@param {type:"string"}
BAMファイルの場所 = '~/Test' #@param {type:"string"}
解析するBAMファイルの名前 = 'test.bam' #@param {type:"string"}
出力先 = '~/Test/output/TSVC' #@param {type:"string"}
出力ファイル名 = 'tsvc.vcf' #@param {type:"string"}
cmd = 'tvc'+\
' --input-bam '+BAMファイルの場所+'/'+解析するBAMファイルの名前+\
' --reference '+リファレンスファイル+\
' --num-threads '+str(スレッド数)+\
' --parameters-file '+パラメータ類のある場所+'/'+パラメータファイル名+\
' --error-motifs-dir '+エラーモチーフのある場所+\
' --output-dir '+出力先+\
' --output-vcf '+出力ファイル名
print(cmd)

tvc --input-bam ~/Test/test.bam --reference ~/Reference/reference.fasta --num-threads 8 --parameters-file ~/Preferences/TSVC/parameters.json --error-motifs-dir ~/Preferences/TSVC --output-dir ~/Test/output/TSVC --output-vcf tsvc.vcf


### Reference  
 >   1. https://assets.thermofisher.com/TFS-Assets/LSG/manuals/MAN0017972_031419_TorrentSuite_5_12_UG_.pdf



## BWAによるマッピング

### インデックス作成

In [5]:
import os
#@title 実行するとコマンド生成
リファレンスファイル = '~/Reference/reference.fasta' #@param {type:"string"}
ディレクトリ = os.path.dirname(リファレンスファイル)
ファイル = os.path.basename(リファレンスファイル)
リファレンス名 = 'testref' #@param {type:"string"}
cmd = 'cd '+ディレクトリ+'\nbwa index -p '+リファレンス名+' ./'+ファイル
print(cmd)

cd ~/Reference
bwa index -p testref ./reference.fasta


### マッピング

In [ ]:
#@title 実行するとコマンド生成
方法 = 'mem' #@param ['aln', 'bwasw', 'mem']
リードの種類 = 'short single' #@param ['short single', 'paired end', 'long read']
リファレンスファイルの場所 = '~/Reference' #@param {type:'string'}
リファレンス名 = 'testref' #@param {type:'string'}
シーケンスデータ１ = '~/Test/seq1.fq' #@param {type:'string'}
シーケンスデータ２ = '~/Test/seq2.fq' #@param {type:'string'}
スプリットリードのマーク = True #@param {type:'boolean'}
出力ファイル保存先 = '~/Test/output/BWA' #@param {type:'string'}
出力ファイル名 = 'mapped' #@param {type:'string'}
スレッド数 = 8 #@param {type:'number'}
cmd = 'cd '+リファレンスファイルの場所
cmd = cmd + '\nbwa '+方法 + ' -t '+str(スレッド数)
if スプリットリードのマーク == True:
  cmd = cmd + ' -M'
cmd = cmd + ' ' + リファレンス名 + ' ' + シーケンスデータ１
if リードの種類 == 'paired end':
  cmd = cmd + ' ' + シーケンスデータ２
cmd = cmd + ' > '+出力ファイル名+'.sam'
cmd = cmd + '\nmkdir -p '+出力ファイル保存先+'\nmv ./'+出力ファイル名+'.sam '+出力ファイル保存先
print(cmd)

cd ~/Reference
bwa mem -t 8 -M testref ~/Test/seq1.fq > mapped.sam
mkdir -p ~/Test/output/BWA
mv ./mapped.sam ~/Test/output/BWA


### Reference
 > 1. http://bio-bwa.sourceforge.net/bwa.shtml

## Samtools （よく使う機能のみ）

### index
*.baiファイルの作成。

In [ ]:
#@title 実行するとコマンド生成
対象のファイルパス = '~/Test/test.bam' #@param {type:'string'}

cmd = 'samtools index'+ ' ' + 対象のファイルパス
print(cmd)

samtools index ~/Test/test.bam


###view
リード情報の表示、部分抽出など。   

In [ ]:
#@title 実行するとコマンド生成
対象のファイルパス = '~/Test/test.bam' #@param {type:'string'}
スレッド数 = 8 #@param {type:'number'}
出力ファイル形式 = 'sam' #@param ['sam','bam','cram']
出力ファイル名 = '~/Test/output/samtools/test' #@param {type:'string'}
ヘッダあり = True #@param {type:'boolean'}
ヘッダ情報のみ取得 = False #@param {type:'boolean'}
リード数のみ取得 = False #@param {type:'boolean'}
リファレンスファイル = '~/Reference/reference.fasta' #@param {type:'string'}
領域指定 = '~/Reference/region.bed' #@param {type:'string'}

cmd = 'samtools view'
cmd = cmd + ' -@' + str(スレッド数) + ' -T ' +リファレンスファイル
if リード数のみ取得 == True:
  cmd = cmd + ' -c '
elif ヘッダ情報のみ取得 == True:
  cmd = cmd + ' -H '
else:
  if ヘッダあり == True:
    cmd = cmd + ' -h'
    if 出力ファイル形式 == 'bam':
      cmd = cmd + ' -b ' + '-o '+出力ファイル名+'.bam'
    elif 出力ファイル形式 == 'cram':
      cmd = cmd + ' -C ' + '-o '+出力ファイル名+'.cram'
    else:
      cmd = cmd + ' -o '+出力ファイル名+'.sam'
  if 領域指定 != '':
    cmd = cmd + ' -L '+領域指定+''
cmd = cmd +  ' ' + 対象のファイルパス  
print(cmd)

samtools view -@8 -T ~/Reference/reference.fasta -h -o ~/Test/output/samtools/test.sam -L ~/Reference/region.bed ~/Test/test.bam


### tview
リード分布の可視化（SSH接続時の簡単な確認用）。  
本格的な図示にはIGVを使う方が良い。

In [ ]:
import re
#@title 実行するとコマンド生成
対象のファイルパス = '~/Test/test.bam' #@param {type:'string'}
表示形式 = 'Curses' #@param ['Curses','HTML','Text']
位置指定 = '' #@param {type:'string'}
cmd = 'samtools tview'
if re.match('\w+:\d+',位置指定):
  cmd = cmd + ' -d ' + 表示形式[0] + ' -p ' + 位置指定 + ' ' + 対象のファイルパス
else:
  cmd = cmd + ' -d ' + 表示形式[0]+ ' ' + ' ' + 対象のファイルパス
print(cmd)

samtools tview -d C  ~/Test/test.bam


### sort
BAMファイルのソート。  
マッピング用のパイプラインスクリプトを使わず、マニュアルでマッピングした場合は個別にかける必要あり。

In [ ]:
#@title 実行するとコマンド生成
対象のファイルパス = '~/Test/test.bam' #@param {type:'string'}
スレッド数 = 8 #@param {type:'number'}
メモリ使用量 = '1000M'  #@param {type:'string'}
出力ファイル形式 = 'sam' #@param ['sam','bam','cram']
出力ファイル名 = '~/Test/output/samtools/test' #@param {type:'string'}

ソート順 = 'Position' #@param ['Position','Name']

cmd = 'samtools sort'
if ソート順 == 'Name':
  cmd = cmd + ' -n'
cmd = cmd + ' -l1 -T tmp -@'+str(スレッド数)+' -m '+メモリ使用量 +\
' -O '+出力ファイル形式 + ' -o '+出力ファイル名 + '.'+出力ファイル形式 + ' '+対象のファイルパス
print(cmd)

samtools sort -l1 -T tmp -@8 -m 1000M -O sam -o ~/Test/output/samtools/test.sam ~/Test/test.bam


### merge
複数回に分けてシーケンスしたデータの結合。  
※ionTorrent系で出力したBAMファイルの場合、catで繋ぐだけだとTorrent suiteに再読み込み出来ない。mergeでもヘッダ情報を書き換える必要があるので注意。  

In [ ]:
#@title 実行するとコマンド生成
対象のファイルパス = '~/Test/test.bam ~/Test/test2.bam' #@param {type:'string'}
出力ファイル名 = '~/Test/output/samtools/test' #@param {type:'string'}

cmd = 'samtools merge'
cmd = cmd + ' -@' + str(スレッド数) +\
' -o '+出力ファイル名 +  '.bam ' + 対象のファイルパス
print(cmd)

samtools merge -@8 -o ~/Test/output/samtools/test.bam ~/Test/test.bam ~/Test/test2.bam


### fastq
BAMをfastqファイルに変換。再マッピングするときなどに使用。

In [ ]:
#@title 実行するとコマンド生成
対象のファイルパス = '~/Test/test.bam' #@param {type:'string'}
リードの種類 = 'single' #@param ['single', 'paired']
出力ファイル名 = '~/Test/output/samtools/test' #@param {type:'string'}

cmd = 'samtools fastq'
if リードの種類 == 'single':
  cmd = cmd + ' '+対象のファイルパス + ' > '+出力ファイル名 + '.fq '
else :
  cmd = cmd + ' -1 '+出力ファイル名 +  '1.fq ' + ' -2 '+出力ファイル名 +  '2.fq -0 /dev/null -s /dev/null ' + 対象のファイルパス
print(cmd)

samtools fastq ~/Test/test.bam > ~/Test/output/samtools/test.fq 


### Reference  
 > 1. http://www.htslib.org/doc/samtools.html

## BCFtools （よく使う機能のみ）

### mpileup & call
マッピング後の small variant call（主にillumina解析時）。  
ionTorrent系でシーケンスした場合は、A/Tの連続領域などでミスが出やすいので、Torrent Suite 付属の variant call を使う方が良い。  

In [ ]:
#@title 実行するとコマンド生成
対象のファイルパス = '~/Test/test.bam' #@param {type:'string'}
出力ファイル形式 = 'vcf' #@param ['vcf','bcf']
出力ファイル名 = '~/Test/output/samtools/test' #@param {type:'string'}
リファレンスファイル = '~/Reference/reference.fasta' #@param {type:'string'}

cmd = 'bcftools mpileup -Ou'+\
' -f '+リファレンスファイル+' '+対象のファイルパス+\
' | bcftools call -vm'
if 出力ファイル形式 == 'vcf':
  cmd = cmd + ' -O z'
else:
  cmd = cmd + ' -O b'
cmd = cmd + ' -o '+出力ファイル名+'.gz'
print(cmd)

bcftools mpileup -Ou -f ~/Reference/reference.fasta ~/Test/test.bam | bcftools call -vm -O z -o ~/Test/output/samtools/test.gz


### Reference
 > 1. http://www.htslib.org/doc/bcftools.html

## Bowtie2によるマッピング

### インデックス作成

In [56]:
import os
#@title 実行するとコマンド生成

リファレンスファイル = '~/Reference/reference.fasta' #@param {type:"string"}
ディレクトリ = os.path.dirname(リファレンスファイル)
ファイル = os.path.basename(リファレンスファイル)
リファレンス名 = 'testref2' #@param {type:"string"}
cmd = 'cd '+ディレクトリ+'\nbowtie2-build -f ./'+ファイル+' '+リファレンス名
print(cmd)

cd ~/Reference
bowtie2-build -f ./reference.fasta testref2


### マッピング

### Reference
 > 1. http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml

## STARによるマッピング for RNA Seq

### インデックス作成

In [59]:
#@title 実行するとコマンド生成
スレッド数 = 8  #@param {type:"number"}
リファレンスファイル = '~/Reference/reference.fasta' #@param {type:"string"}
ディレクトリ = os.path.dirname(リファレンスファイル)
ファイル = os.path.basename(リファレンスファイル)
リファレンス名 = 'testref3' #@param {type:"string"}
cmd = 'cd '+ディレクトリ+'\nSTAR --runMode genomeGenerate --genomeDir '+ リファレンス名+' --genomeFastaFiles ./'+ファイル
print(cmd)

cd ~/Reference
STAR --runMode genomeGenerate --genomeDir testref3 --genomeFastaFiles ./reference.fasta


### マッピング

### Reference
 > 1. https://raw.githubusercontent.com/alexdobin/STAR/master/doc/STARmanual.pdf

## MACS2によるピーク検出 for ChIP Seq

In [4]:
#@title 実行するとコマンド生成
BAMファイルパス = '~Test/test.bam' #@param {type:"string"}
コントロールファイルパス  = '~Test/control.bam' #@param {type:"string"}
ゲノムスケール = 'ce' #@param['hs', 'mm', 'dm', 'ce', 'custom']
ゲノムサイズ = 1.0e8 #@param {type:"number"}
重複リード = '利用しない' #@param['利用しない', '利用する']
ピーク特性 = 'broad' #@param['broad', 'narrow']
p値 = 0 #@param {type:"number"}
出力ファイル名 = 'peaks' #@param {type:"string"}

cmd = 'macs2 callpeak'+\
' -t '+BAMファイルパス+\
' -c '+コントロールファイルパス+\
' -f BAM'
if ゲノムスケール == 'custom':
  cmd = cmd + ' -g '+ str(ゲノムサイズ)
else:
  cmd = cmd + ' -g '+ ゲノムスケール 
if 重複リード == '利用する':
  cmd = cmd + ' -keep-dup'
if ピーク特性 == 'broad':
  cmd = cmd + ' --broad'
if p値 > 0:
  cmd = cmd + '-p '+str(p値)
cmd = cmd + ' --nomodel -n '+出力ファイル名
print(cmd)

macs2 callpeak -t ~Test/test.bam -c ~Test/control.bam -f BAM -g ce --broad --nomodel -n peaks
